In [2]:
import os
import random

# get the file names in each directory
train_neg = [f for f in os.listdir('aclImdb/train/neg')]
train_pos = [f for f in os.listdir('aclImdb/train/pos')]
test_neg = [f for f in os.listdir('aclImdb/test/neg')]
test_pos = [f for f in os.listdir('aclImdb/test/pos')]

def read(directory,path):
    texts = []
    for f in directory:
        f=os.path.join(path,f)
        with open(f, "rt") as inp_file:
            text = inp_file.readlines()
            assert len(text)==1
            texts.append(text[0].strip())
    return texts

# read all the files in each directory into a list
train_neg = read(train_neg,'aclImdb/train/neg')
train_pos = read(train_pos,'aclImdb/train/pos')
test_neg = read(test_neg,'aclImdb/test/neg')
test_pos = read(test_pos,'aclImdb/test/pos')

# add labels for texts
train_neg = [(text,-1) for text in train_neg]
train_pos = [(text,1) for text in train_pos]
test_neg = [(text,-1) for text in test_neg]
test_pos = [(text,1) for text in test_pos]

# shuffle the training set
train = train_neg + train_pos
random.shuffle(train)

train_texts = [text for text, label in train]
train_labels = [label for text, label in train]
#print(train_texts[:3])
print(train_labels[:20])


[1, -1, 1, 1, 1, 1, -1, -1, 1, -1, 1, -1, 1, -1, 1, 1, -1, -1, 1, -1]


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix
from eli5 import show_weights

space_tokenizer = lambda text: text.split()


# Featurization and vectorization
vectorizer = TfidfVectorizer(tokenizer=space_tokenizer, ngram_range=(1,2))
vectorizer.fit(train_texts)

train_X = vectorizer.transform(train_texts)
#devel_X = vectorizer.transform(devel_texts)
test_X = vectorizer.transform(test_texts)


NameError: name 'test_texts' is not defined

In [6]:
%%time
# Training and prediction

classifier = LinearSVC(
    C=1.0,
    class_weight=None,
    max_iter=1000,
    loss='squared_hinge'
)
classifier.fit(train_X, train_labels)



CPU times: user 2.88 s, sys: 76 ms, total: 2.96 s
Wall time: 2.96 s


In [7]:
test = test_neg + test_pos
test_texts = [text for text, label in test]
test_labels = [label for text, label in test]
test_X = vectorizer.transform(test_texts)

In [8]:
%%time
pred_labels = classifier.predict(test_X)

CPU times: user 52 ms, sys: 0 ns, total: 52 ms
Wall time: 53 ms


In [9]:
# Evaluation and analysis
accuracy = accuracy_score(test_labels, pred_labels)

tn, fp, fn, tp = confusion_matrix(test_labels, pred_labels, labels=[-1, 1]).ravel()
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f_score = 2 * precision * recall / (precision + recall)

print('accuracy {:.2%}'.format(accuracy))
print('precision {:.2%}, recall {:.2%}, f-score {:.2%}'.format(precision, recall, f_score))


accuracy 89.72%
precision 89.37%, recall 90.15%, f-score 89.76%


In [10]:
show_weights(classifier, vec=vectorizer)
